In [ ]:
import json
import nltk
import pandas as pd
import random
import nltk

nltk.download('punkt')
nltk.download('reuters')
from nltk.corpus import reuters
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
from nltk.stem import PorterStemmer
from gensim.models import Word2Vec


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package reuters to /root/nltk_data...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
def filter_stop_words(words):
    stop_words = set(stopwords.words('english'))
    filtered_words = [word for word in words if word.lower() not in stop_words]
    return filtered_words

In [ ]:
def lemmatize_words(words):
    lemmatizer = WordNetLemmatizer()
    lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
    return lemmatized_words

In [ ]:
path='/content/gdrive/MyDrive/Sarcasm_Headlines_Dataset.json'
raw_data_json = pd.read_json(path, lines=True)

print("Raw data is :\n\n", raw_data_json.loc[:20,['is_sarcastic','headline']])

Raw data is :

     is_sarcastic                                           headline
0              1  thirtysomething scientists unveil doomsday clo...
1              0  dem rep. totally nails why congress is falling...
2              0  eat your veggies: 9 deliciously different recipes
3              1  inclement weather prevents liar from getting t...
4              1  mother comes pretty close to using word 'strea...
5              0                               my white inheritance
6              0         5 ways to file your taxes with less stress
7              1  richard branson's global-warming donation near...
8              1  shadow government getting too large to meet in...
9              0                 lots of parents know this scenario
10             0  this lesbian is considered a father in indiana...
11             0  amanda peet told her daughter sex is 'a specia...
12             0  what to know regarding current treatments for ...
13             0  chris christie

In [ ]:
X = []
Y = []
special_characters = "!@#$%^&*()-_+=[]{}|:;\"'<>,.?/"

for index, elem in raw_data_json.iterrows():
  text = elem['headline']
  outcome = elem['is_sarcastic']

  x=[]
  str1 = ""
  for char in (text + ' '):
    if char == ' ' or char == '-' or char == '_':
      x.append(str1)
      str1 = ""
    elif char not in special_characters:
      str1 = str1 + char

  if len(x) > 0:
    filtered_x = filter_stop_words(x)
  else:
    continue
  if len(filtered_x) > 0:
    lemmatized_x = lemmatize_words(filtered_x)
    X.append(lemmatized_x)
    Y.append(outcome)


In [ ]:
assert (len(X) == len(Y))

sarcastic_data = []
non_sarcastic_data = []

n_X = len(X)

for i in range(n_X):
  if Y[i] == 1:
    sarcastic_data.append(X[i])
  else:
    non_sarcastic_data.append(X[i])

print("Number of sarcastic samples : " + str(len(sarcastic_data)) + "\nNumber of non sarcastic samples : " + str(len(non_sarcastic_data)))


seed_value = 42
split_at = 9544 # decides number of sarcastic and non-sarcastic data in train set
n_sarcastic = len(sarcastic_data)
n_non_sarcastic = len(non_sarcastic_data)
random.seed(seed_value)



train_data = sarcastic_data[:split_at] + non_sarcastic_data[:split_at]
train_labels = ([1] * split_at) + ([0] * split_at)
test_data = (sarcastic_data[split_at:] + non_sarcastic_data[split_at:])
test_labels = ([1] * (n_sarcastic - split_at)) + ([0] * (n_non_sarcastic -split_at))

n_train = len(train_data)
n_test = len(test_data)

assert (len(train_data) == len(train_labels))
assert (len(test_data) == len(test_labels))

#combined_train_data = list(zip(train_data, train_labels))
combined_train_data = [[x, y] for x, y in zip(train_data, train_labels)]
# combined_test_data = list(zip(test_data, test_labels))
combined_test_data = [[x, y] for x, y in zip(test_data, test_labels)]
random.shuffle(combined_train_data)
random.shuffle(combined_test_data)

# X_train, Y_train = zip(*combined_train_data) # Training set
# X_test, Y_test = zip(*combined_test_data) # Test set

X_train = [item[0] for item in combined_train_data]
Y_train = [item[1] for item in combined_train_data]

X_test = [item[0] for item in combined_test_data]
Y_test = [item[1] for item in combined_test_data]


print("Length of training set is : ", n_train)
print("Length of test set is : ", n_test)

for i in range(10):
  print(X_train[i], Y_train[i])

print(type(X_train))

Number of sarcastic samples : 13634
Number of non sarcastic samples : 14984
Length of training set is :  19088
Length of test set is :  9530
['christmas', 'pageant', 'enters', 'pre', 'production'] 1
['unidentified', 'yowling', 'animal', 'carrier', 'apparently', 'named', 'kiwi'] 1
['study', 'every', '10', 'second', 'skyscraper', 'window', 'washer', 'fall', 'death'] 1
['paranoid', 'oscar', 'pistorius', 'still', 'think', 'burglar'] 1
['new', 'war', 'enables', 'mankind', 'resolve', 'disagreement'] 1
['strange', 'looking', 'pup', 'find', 'perfect', 'family'] 0
['mlb', 'player', 'yordano', 'ventura', 'andy', 'marte', 'die', 'separate', 'car', 'crash'] 0
['science', 'behind', 'celebrity', 'like', 'ryan', 'lochte', 'tell', 'fib'] 0
['voice', 'recognition', 'software', 'yelled'] 1
['flooding', 'texas', 'lead', 'mosquito', 'borne', 'illness'] 0
<class 'list'>


In [ ]:
headlines = reuters.sents(categories='acq')
model = Word2Vec(sentences=(X_train + X_test), vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
for i in range(n_train):
  encodedData = []
  for word in X_train[i]:
    encodedData.append(model.wv[word])
 # print("encdata :", encodedData)
  X_train[i] = encodedData

for i in range(n_test):
  encodedData = []
  for word in X_test[i]:
    encodedData.append(model.wv[word])
  X_test[i] = encodedData

print("Encoded elements are : ", X_train[0], X_test[0])

Encoded elements are :  [array([-0.33134785,  0.6095842 ,  0.32779014,  0.09108899,  0.01526477,
       -0.8061468 ,  0.14279924,  0.8844124 , -0.21062651, -0.3705784 ,
       -0.14857729, -0.8610757 , -0.14940709,  0.14554106,  0.12260959,
       -0.41819417,  0.07526311, -0.4789979 , -0.13803518, -0.9562892 ,
        0.20817663,  0.23468441,  0.5745352 , -0.1706367 , -0.11422875,
        0.10155062, -0.20094897, -0.3410442 , -0.5517176 ,  0.0137904 ,
        0.49209812, -0.07207744, -0.00981681, -0.24172151,  0.02763076,
        0.35334942,  0.00829452, -0.29157498, -0.21747015, -0.6990117 ,
        0.12799348, -0.49788547, -0.0931904 ,  0.08500276,  0.2530436 ,
       -0.29344356, -0.39321733,  0.10955852, -0.03437136,  0.4220828 ,
        0.14511748, -0.40769666, -0.2724192 , -0.13325106, -0.48107424,
        0.2285085 ,  0.44176137, -0.0139594 , -0.60685205,  0.02279072,
        0.0166795 ,  0.01805042, -0.23566881,  0.134385  , -0.35799232,
        0.48499098,  0.16130674,  0.308